# Tugas Prapraktikum

Tugas Prapraktikum dikerjakan dengan _dataset_ [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package/download?datasetVersionNumber=2). Tanpa meninjau waktu (`date`), prediksi status hujan pada keesokan harinya (`RainTomorrow`). Berikan nilai `1` jika diprediksi hujan pada keesokan harinya, `0` jika tidak.

<br>
Tugas dikerjakan secara berkelompok. Setiap kelompok terdiri atas 2 (dua) mahasiswa. Kumpulkan paling lambat pada Minggu, 16 April 2023, pukul 23:59 WIB melalui Edunex.

# 0. Persiapan Data and Pustaka

In [5]:
# Letakkan pustaka di sini.
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype as numeric
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from multiprocessing import Process

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score,f1_score,accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, ShuffleSplit

In [6]:
# Baca data di sini.
data = pd.read_csv('weatherAUS.csv', usecols=lambda col: col not in ['Date'])
df = pd.DataFrame(data)
df.copy()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,ENE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,N,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,WNW,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,N,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


Menampilkan type setiap kolom

In [93]:
df.dtypes

Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

Menampilkan seluruh data dari setiap fitur

In [126]:
df.loc[:, df.columns != "RainTomorrow"]

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,ENE,...,11.0,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No
145456,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,N,...,9.0,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No
145457,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,WNW,...,9.0,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No
145458,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,N,...,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No


# I. Pemahaman Data
Tujuan dari bagian ini adalah peserta dapat memahami kualitas dari data yang diberikan. Hal yang diliputi adalah sebagai berikut:
1. Ukuran data
2. Statistik dari tiap fitur
3. Pencilan (_outlier_)
4. Korelasi
5. Distribusi 

## I.1 
Carilah:
1. Ukuran dari data (instansi dan fitur)
2. Tipe dari setiap fitur 
3. Banyak nilai unik dari fitur yang bertipe kategorikal
4. Nilai minimum, maksimum, rata-rata, median, dan standar deviasi dari fitur nonkategorikal

In [141]:
# I.1 Kode di sini.
print("1. Ukuran dari data (instansi dari fitur)\n")
print("Ukuran data: " + str(df.shape[0])) # ukuran data
print("Banyak fitur: " + str(df.loc[:, df.columns != "RainTomorrow"].shape[1])) # banyak fitur

print("\n2. Tipe dari setiap fitur")
print('''
    - Location: Nominal/Kategorikal
    - MinTemp: Numeric (continuous)
    - MaxTemp: Numeric (continuous)
    - Rainfall: Numeric (continuous)
    - Evaporation: Numeric (continuous)
    - Sunshine: Numeric (continuous)
    - WindGustDir: Nominal/Kategorikal
    - WindGustSpeed: Numeric (continuous)
    - WindDir9am: Nominal/Kategorikal
    - WindDir3pm: Nominal/Kategorikal
    - WindSpeed9am: Numeric (continuous)
    - WindSpeed3pm: Numeric (continuous)
    - Humidity9am: Numeric (continuous)
    - Humidity3pm: Numeric (continuous)
    - Pressure9am: Numeric (continuous)
    - Pressure3pm: Numeric (continuous)
    - Cloud9am: Numeric (continuous)
    - Cloud3pm: Numeric (continuous)
    - Temp9am: Numeric (continuous)
    - Temp3pm: Numeric (continuous)
    - RainToday: Binary (Yes/No)
''')


print("3. Banyak nilai unik dari fitur yang bertipe kategorikal")
print("Jumlah nilai unik untuk feature Location: ", df["Location"].nunique())
print("Jumlah nilai unik untuk feature WindGustDir: ", df["WindGustDir"].nunique())
print("Jumlah nilai unik untuk feature WindDir9am: ", df["WindDir9am"].nunique())


print("4. Nilai minimum, maksimum, rata-rata, median, dan standar deviasi dari fitur nonkategorikal")



1. Ukuran dari data (instansi dari fitur)

Ukuran data: 145460
Banyak fitur: 22

2. Tipe dari setiap fitur

    - Location: Nominal/Kategorikal
    - MinTemp: Numeric (continuous)
    - MaxTemp: Numeric (continuous)
    - Rainfall: Numeric (continuous)
    - Evaporation: Numeric (continuous)
    - Sunshine: Numeric (continuous)
    - WindGustDir: Nominal/Kategorikal
    - WindGustSpeed: Numeric (continuous)
    - WindDir9am: Nominal/Kategorikal
    - WindDir3pm: Nominal/Kategorikal
    - WindSpeed9am: Numeric (continuous)
    - WindSpeed3pm: Numeric (continuous)
    - Humidity9am: Numeric (continuous)
    - Humidity3pm: Numeric (continuous)
    - Pressure9am: Numeric (continuous)
    - Pressure3pm: Numeric (continuous)
    - Cloud9am: Numeric (continuous)
    - Cloud3pm: Numeric (continuous)
    - Temp9am: Numeric (continuous)
    - Temp3pm: Numeric (continuous)
    - RainToday: Binary (Yes/No)

3. Banyak nilai unik dari fitur yang bertipe kategorikal
Jumlah nilai unik untuk feature Lo

## I.2
Carilah:
1. Nilai hilang (_missing_) dari setiap fitur
2. Nilai pencilan (_outlier_) dari setiap fitur

Missing Value

In [7]:
# I.2 Kode di sini.
#Jumlah missing value dari setiap fitur :
missing_values = df.drop(columns="RainTomorrow").isna().sum()
print(f"Jumlah missing value dari setiap fitur : \n {missing_values}\n")

#Missing value dari setiap fitur :
for column in df.columns:
  if (column != "RainTomorrow"):
    print("• "+column+str(": ")+str(len(df[df[column].isna()])))
    print("\n", df[df.isna().any(axis=1)])

Jumlah missing value dari setiap fitur : 
 Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
dtype: int64

• Location: 0

        Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0        Albury     13.4     22.9       0.6          NaN       NaN   
1        Albury      7.4     25.1       0.0          NaN       NaN   
2        Albury     12.9     25.7       0.0          NaN       NaN   
3        Albury      9.2     28.0       0.0          NaN       NaN   
4        Albury     17.5     32.3       1.0          NaN       NaN   
...         ...      ...

Outliers
Outliers in MinTemp
Number of outliers: 54

Outliers in MaxTemp
Number of outliers: 489

Outliers in Rainfall
Number of outliers: 25578

Outliers in Evaporation
Number of outliers: 1995

Outliers in Sunshine
[]
Number of outliers: 0



Outliers in WindGustSpeed
Number of outliers: 3092

Outliers in WindSpeed9am
Number of outliers: 1817

Outliers in WindSpeed3pm
Number of outliers: 2523

Outliers in Humidity9am
Number of outliers: 1425

Outliers in Humidity3pm
Number of outliers: 0

Outliers in Pressure9am
Number of outliers: 1191

Outliers in Pressure3pm
Number of outliers: 919

Outliers in Cloud9am
Number of outliers: 0

Outliers in Cloud3pm
Number of outliers: 0

Outliers in Temp9am
Number of outliers: 262

Outliers in Temp3pm
Number of outliers: 764

In [42]:
numeric_cols = df.select_dtypes(include="number").columns
for column in numeric_cols:
    # Calculate IQR for the column
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Calculate the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers in the column
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    # Print the number of outliers and their values
    print("Outliers in", column)
    print(outliers[column].values)
    print("Number of outliers:", len(outliers))

Outliers in MinTemp
[-6.9 -6.5 -8.  -6.7 -6.6 -6.8 -7.6 -6.8 -7.  -6.9 -7.  -6.7 -8.2 -7.
 -6.5 -6.7 -8.5 -8.  -7.  -7.1 -6.4 -6.5 -6.5 -6.6 -6.6 -6.4 -6.5 -6.7
 -6.4 -7.6 -6.6 -7.5 -7.  -6.4 -6.8 -6.4 -6.7 -6.5 -8.2 -7.2 -6.5 -7.8
 -7.  -7.5 -7.3 -7.  31.9 33.9 31.4 31.4 31.4 31.8 31.2 31. ]
Number of outliers: 54
Outliers in MaxTemp
[44.8 43.7 45.8 45.1 44.4 44.4 46.4 43.9 45.4 44.3 43.7 46.  43.8 44.3
 45.2 45.7 46.6 47.3 43.9 44.5 47.3 44.1 44.  46.5 44.9 44.8 43.9 45.1
 44.9 46.9 46.4 45.3 44.5 43.7 44.4 43.8 44.6 47.  45.8 46.4 45.2 44.1
 45.  43.7 44.8 45.5 -1.7  0.   0.   2.4  2.3  0.1 -0.1 -2.2 -0.6  0.6
  2.2  0.4 -0.1 -0.7 -1.8 -0.2  0.1  0.9 -1.3  0.1  1.8 -0.6 -0.6  1.6
  0.3  1.2 -0.5  1.   0.5  0.3  0.2  1.9  2.   2.   2.4  0.9  2.2 -2.4
 -0.3  0.1  0.6 -2.5 -0.7  2.1  1.4  1.8  1.   2.1  1.4  2.2 -2.7 -1.
  0.4  2.1  0.6  2.1  1.   1.1  2.1 -1.7  0.   0.5  1.3  1.6 -1.4  0.3
  2.4  0.1  0.1 -0.8 -2.   2.2 -1.3 -0.4  1.7 -0.9  2.1 -2.1 -1.5 -0.6
 -3.1 -1.3  1.1  0.4 -1.2

## I.3
Lakukan:
1. Pencarian korelasi antarfitur
2. Visualisasi distribusi setiap fitur (kategorikal dan kontinu)
3. Visualisasi distribusi setiap fitur per target (`RainTomorrow`)

Korelasi setiap fitur

In [ ]:
# I.3 Kode di sini.
df.corr()

Visualisasi distribusi setiap fitur

In [ ]:

# Visualisasi distribusi kategorikal fitur
kategorikal = ["Location", "WindGustDir", "WindDir9am", "WindDir3pm", "RainToday"]
for feature in kategorikal:
    plt.figure()
    df[feature].value_counts().plot(kind="bar")
    plt.title(feature)

# Visualisasi distribusi kontinu fitur
kontinu = ["MinTemp", "MaxTemp", "Rainfall", "Evaporation", "Sunshine", "WindGustSpeed",
                       "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm", "Pressure9am",
                       "Pressure3pm", "Temp9am", "Temp3pm", "Cloud9am", "Cloud3pm"]
for feature in kontinu:
    plt.figure()
    df[feature].plot(kind="hist", bins=50)
    plt.title(feature)

plt.show()

In [45]:
import seaborn as sns
for feature in df.columns:
    if feature != 'RainTomorrow':
        sns.displot(data=df, x=feature, hue='RainTomorrow', kde=True)

## I.4
Lakukanlah analisis lebih lanjut jika diperlukan, kemudian lakukan hal berikut:
1. Penambahan fitur jika memungkinkan
2. Pembuangan fitur yang menurut kalian tidak dibutuhkan
3. Penanganan nilai hilang
4. Transformasi data kategorikal menjadi numerikal (_encoding_)
5. _Scaling_ dengan `MinMaxScaler`

Penambahan dan Pengurangan fitur


Melakukan penambahan fitur cuaca seperti suhu titik embun dan kecepatan angin rata-rata.

Suhu titik embun adalah suhu di mana uap air dalam udara mulai membentuk titik-titik air yang terlihat seperti embun sendiri adalah dew point. Jadi, semakin tinggi angka dew_point_approximation, semakin dekat suhu udara dengan suhu di mana embun mulai terbentuk. 

Selain itu didapatkan juga rumus dari dew_point adalah :
dew_point_approx = (b * gamma(T, RH)) / (c - gamma(T, RH))
dengan gamma(T, RH) = (a * T / (b + T)) + ln(RH/100.0)

Namun, terdapat beberapa nilai yang belum diketahui, sehingga perlu penelitian lebih lanjut untuk menambahkan fitur tersebut.


Sedangkan kecepatan angin rata-rata dapat mempermudah dapat memberikan informasi tambahan untuk menganalisis kemungkinan hujan.

Jadi, yang memungkinkan menurut kami adalah menambahkan informasi mengenai rata-rata dari kecepatan angin yang telah diberikan pada dataset tersebut. 

Sumber : https://www.merdeka.com/jatim/perubahan-cuaca-dipengaruhi-oleh-beberapa-faktor-ini-dia-penjelasan-lengkapnya-kln.html
http://iklim.sumbar.bmkg.go.id/?page_id=184


Mengenai pengurangan fitur, setelah melakukan analisis singkat kami merasa semua fitur diperlukan, namun diperlukan penelitian lebih lanjut apakah diperlukan data yang dihapus. Namun untuk sementara ini menurut kami, tidak ada fitur yang perlu dihapuskan.
 
Namun berdasarkan data didapatkan outliers sebanyak
Outliers in MinTemp
Number of outliers: 54

Outliers in MaxTemp
Number of outliers: 489

Outliers in Rainfall
Number of outliers: 25578

Outliers in Evaporation
Number of outliers: 1995

Outliers in Sunshine
[]
Number of outliers: 0



Outliers in WindGustSpeed
Number of outliers: 3092

Outliers in WindSpeed9am
Number of outliers: 1817

Outliers in WindSpeed3pm
Number of outliers: 2523

Outliers in Humidity9am
Number of outliers: 1425

Outliers in Humidity3pm
Number of outliers: 0

Outliers in Pressure9am
Number of outliers: 1191

Outliers in Pressure3pm
Number of outliers: 919

Outliers in Cloud9am
Number of outliers: 0

Outliers in Cloud3pm
Number of outliers: 0

Outliers in Temp9am
Number of outliers: 262

Outliers in Temp3pm
Number of outliers: 764
Didapatkan juga jumlah missing value : 
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261

Dengan demikian kami menyimpulkan Evaporation,Sunshine, Cloud9am,Cloud3pm tidak akan berpengaruh karena banyak missing value. Selain itu rainfall juga kami hapus karena outliers yang sangat besar dan akan tidak mempengaruhi.

Kode untuk menambahkan fitur rata-rata kecepatan angin

In [ ]:
# I.4 Put your code here


# Menambahkan fitur rata-rata kecepatan angin
df.insert(df.columns.get_loc('WindSpeed3pm')+1, 'AvgWindSpeed', (df['WindGustSpeed'] + df['WindSpeed9am'] + df['WindSpeed3pm']) / 3)
df = df.reindex(columns=['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindSpeed9am', 'WindDir3pm', 'WindSpeed3pm', 'AvgWindSpeed', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow'])
df.head()

#Menghapus fitur Evaporation,Sunshine,Cloud9am,cloud3pm, dan rainfall
columns_to_drop = ['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm', 'Rainfall']
df = df.drop(columns_to_drop, axis=1)
#CEK
df.head()

# Penanganan missing value dengan meng-assign nilai rata-rata untuk feature bertipe numerikal dan
# meng-assign modus atau nilai yang paling sering muncul untuk feature bertipe kategorikal

# Mengisi missing value pada kolom numerik dengan rata-rata
numeric_colums = df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_colums:
    df[col].fillna(df[col].mean(), inplace=True)

# Mengisi missing value pada kolom nominal dengan modus
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindSpeed9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Albury,13.4,22.9,0.6,5.468232,7.611178,W,44.0,W,20.0,...,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,No,No
1,Albury,7.4,25.1,0.0,5.468232,7.611178,WNW,44.0,NNW,4.0,...,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,No,No
2,Albury,12.9,25.7,0.0,5.468232,7.611178,WSW,46.0,W,19.0,...,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,No,No
3,Albury,9.2,28.0,0.0,5.468232,7.611178,NE,24.0,SE,11.0,...,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,No,No
4,Albury,17.5,32.3,1.0,5.468232,7.611178,W,41.0,ENE,7.0,...,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,No,No


Encoding Kategorikal Fitur dan target rain tomorrow

In [9]:
kategorikal = ["Location", "WindGustDir", "WindDir9am", "WindDir3pm", "RainToday","RainTomorrow"]

# loop untuk setiap kolom kategorikal dan melakukan encoding
for col in kategorikal:
    # inisialisasi label encoder
    le = LabelEncoder()
    # encoding kolom kategorikal
    df[col] = le.fit_transform(df[col].astype(str))
#Melakukan pemeriksaan hasil
df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindSpeed9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2,13.4,22.9,0.6,5.468232,7.611178,13,44.0,13,20.0,...,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,0,0
1,2,7.4,25.1,0.0,5.468232,7.611178,14,44.0,6,4.0,...,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,0,0
2,2,12.9,25.7,0.0,5.468232,7.611178,15,46.0,13,19.0,...,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,0,0
3,2,9.2,28.0,0.0,5.468232,7.611178,4,24.0,9,11.0,...,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,0,0
4,2,17.5,32.3,1.0,5.468232,7.611178,13,41.0,1,7.0,...,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,0,0


Scaling dengan MinMaxScaler

In [10]:
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df)

# II. Desain Eksperimen
Tujuan dari bagian ini adalah peserta dapat memahami cara melakukan eksperimen mencari metode terbaik dengan benar. Hal yang diliputi adalah sebagai berikut:
1. Pembuatan model
2. Proses validasi
3. _Hyperparameter tuning_

## II.1
Tentukanlah metrik yang akan digunakan pada eksperimen kali ini. Metrik yang dapat lebih dari satu jenis.

Berikut ini metrik yang digunakan pada eksperimen kali ini, yang merupakan classification metrik :
1. Accuracy Score
2. Recall Score
3. F1 Score 
4. Precisision Score

## II.2 
Bagi data dengan perbandingan 0,8 untuk data latih dan 0,2 untuk data validasi.

In [16]:

# II.2 Kode di sini
x= df.loc[:, df.columns!='RainTomorrow']
y= df['RainTomorrow']
# Bagi dataset menjadi data training dan validasi dengan proporsi 80:20
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_1, x_val_1, y_train_1, y_val_1 = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_2, x_val_2, y_train_2, y_val_2 = train_test_split(x, y, test_size=0.2, random_state=42)

print("Xval : ", x_val.shape[0])
print("Yval : ", y_val.shape[0])
print("Persentase Xval: ", x_val.shape[0]/df.shape[0])
print("Persentase Yval: ", y_val.shape[0]/df.shape[0])

print("Xtrain : ", x_train.shape[0])
print("Ytrain : ", y_train.shape[0])
print("Persentase Xtrain: ", x_train.shape[0]/df.shape[0])
print("Persentase Ytrain: ", y_train.shape[0]/df.shape[0])

Xval :  29092
Yval :  29092
Persentase Xval:  0.2
Persentase Yval:  0.2
Xtrain :  116368
Ytrain :  116368
Persentase Xtrain:  0.8
Persentase Ytrain:  0.8


In [159]:
print(y_train)

111548    1
142528    0
65304     0
40651     0
97487     0
         ..
110268    0
119879    0
103694    0
131932    0
121958    0
Name: RainTomorrow, Length: 116368, dtype: int32


## II.3
Lakukan hal berikut:
1. Prediksi dengan menggunakan model _logistic regression_ sebagai _baseline_.
2. Tampilkan evaluasi dari model yang dibangun dari metrik yang ditentukan pada II.1
3. Tampilkan _confusion matrix_.

In [164]:
# II.3 Kode di sini
logreg = LogisticRegression(random_state=0, max_iter=1000).fit(x_train, y_train)
logreg_prediction = logreg.predict(x_val)


Hasil prediksi dan aktual dataset dengan Logistic Regression

In [165]:
print(list(logreg_prediction))
print()
print("Hasil aktual dari dataset: ")
print(list(y_val))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 

Classification report hasil prediksi

In [166]:
print(classification_report(y_val, logreg_prediction))
print()

print("Nilai dari precision hasil prediksi :", precision_score(y_val, logreg_prediction))
print("Nilai dari recall hasil prediksi :", recall_score(y_val, logreg_prediction))
print("Nilai dari f1 score hasil prediksi :", f1_score(y_val, logreg_prediction))
print("Dengan accuracy hasil prediksi :",accuracy_score(y_val, logreg_prediction))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90     22672
           1       0.71      0.47      0.56      6420

    accuracy                           0.84     29092
   macro avg       0.79      0.71      0.73     29092
weighted avg       0.83      0.84      0.83     29092


Nilai dari precision hasil prediksi : 0.7110952720361131
Nilai dari recall hasil prediksi : 0.4661993769470405
Nilai dari f1 score hasil prediksi : 0.5631762160127951
Dengan accuracy hasil prediksi : 0.840402859892754


Confusion Matrix

In [167]:
print(confusion_matrix(y_val, logreg_prediction))

[[21456  1216]
 [ 3427  2993]]


## II.4 
Lakukanlah:
1. Pembelajaran dengan model lain
2. _Hyperparameter tuning_ untuk model yang dipakai dengan menggunakan _grid search_ (perhatikan _random factor_ pada beberapa algoritma model)
3. Validasi dengan _cross validation_


In [17]:
# II.4 Kode di sini.

1. Pembelajaran dengan model SVM
svc = SVC()
svc.fit(x_train, y_train)
titanic_SVM_pred = svc.predict(x_val)
print("Evaluasi pembelajaran SVM dengan metrics II.1: \n")
print(classification_report(y_val, titanic_SVM_pred))
print("precision hasil prediksi SVM :", precision_score(y_val, titanic_SVM_pred))
print("recall hasil prediksi SVM :", recall_score(y_val, titanic_SVM_pred))
print("f1 score hasil prediksi SVM :", f1_score(y_val, titanic_SVM_pred))
print("Accuracy hasil prediksi SVM :",accuracy_score(y_val, titanic_SVM_pred))
print("\nConfusion matrix: \n")
print(pd.DataFrame(confusion_matrix(y_val, titanic_SVM_pred),columns=['Predicted No','Predicted Yes'],index=['Actual No','Actual Yes']))

def learn_evaluate_model(x_train, y_train, x_val, y_val):
    svc = SVC()
    svc.fit(x_train, y_train)
    titanic_SVM_pred = svc.predict(x_val)
    print("Evaluasi pembelajaran SVM dengan metrics II.1: \n")
    print(classification_report(y_val, titanic_SVM_pred))
    print("precision hasil prediksi SVM :", precision_score(y_val, titanic_SVM_pred))
    print("recall hasil prediksi SVM :", recall_score(y_val, titanic_SVM_pred))
    print("f1 score hasil prediksi SVM :", f1_score(y_val, titanic_SVM_pred))
    print("Accuracy hasil prediksi SVM :",accuracy_score(y_val, titanic_SVM_pred))
    print("\nConfusion matrix: \n")
    print(pd.DataFrame(confusion_matrix(y_val, titanic_SVM_pred),columns=['Predicted No','Predicted Yes'],index=['Actual No','Actual Yes']))

# buat dua proses yang masing-masing menjalankan fungsi learn_evaluate_model untuk dua set data yang berbeda
p1 = Process(target=learn_evaluate_model, args=(x_train_1, y_train_1, x_val_1, y_val_1))
p2 = Process(target=learn_evaluate_model, args=(x_train_2, y_train_2, x_val_2, y_val_2))

# jalankan kedua proses tersebut
p1.start()
p2.start()

# tunggu hingga kedua proses selesai
p1.join()
p2.join()

# 2. Hyperparameter tuning
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
svm_after_tuning = GridSearchCV(svc, param_grid, refit = True, verbose = 3)
svm_after_tuning.fit(x_train, y_train)
titanic_SVM_after_tuning_pred = svm_after_tuning.predict(x_val) 
print("Evaluasi pembelajaran SVM after tuning dengan metrics II.1:\n")
print(classification_report(y_val, titanic_SVM_after_tuning_pred))
print("precision hasil prediksi SVM setelah tuning :", precision_score(y_val, titanic_SVM_after_tuning_pred))
print("recall hasil prediksi SVM setelah tuning :", recall_score(y_val, titanic_SVM_after_tuning_pred))
print("f1 score hasil prediksi SVM setelah tuning :", f1_score(y_val, titanic_SVM_after_tuning_pred))
print("Accuracy hasil prediksi SVM setelah tuning :",accuracy_score(y_val, titanic_SVM_after_tuning_pred))

# 3. Cross validation
crossval_score = cross_val_score(svc, x_train, y_train.values.ravel(), cv=5)
print(f"\nCross validation score: {crossval_score}")

# III. Improvement
Pada bagian ini, kalian diharapkan dapat:
1. melakukan pelatihan dengan data hasil _oversampling_ / _undersampling_, disertai dengan validasi yang benar; serta
2. menerapkan beberapa metode untuk menggabungkan beberapa model.

Kedua hal ini adalah contoh metode untuk meningkatkan kinerja dari model.

## III.1
Lakukanlah:
1. _Oversampling_ pada kelas minoritas pada data latih
2. _Undersampling_ pada kelas mayoritas pada data latih

Pada setiap tahap, latih dengan model *baseline* (II.3), dan validasi dengan data validasi. Data latih dan validasi adalah data yang disusun pada bagian II.2.

In [161]:
# III.1 Kode di sini.

# Melakukan oversampling pada kelas minoritas data latih
ros = RandomOverSampler(sampling_strategy="not majority", random_state=0)
over_x_train, over_x_train = ros.fit_resample(x_train, y_train)

# Melakukan training dengan model baseline 
logistic_reg = LogisticRegression(random_state=0, max_iter=1000).fit(over_x_train, over_x_train)
pred = logistic_reg.predict(x_val)

print(classification_report(y_val, pred))
print("Nilai precision:", precision_score(y_val, pred))
print("Nilai recall:", recall_score(y_val, pred))
print("Nilai F1 score:", f1_score(y_val, pred))
print("Nilai accuracy:",accuracy_score(y_val, pred))

In [ ]:
rus = RandomUnderSampler(sampling_strategy=1, random_state=0)
under_x_train, under_x_train = rus.fit_resample(x_train, x_train)

# Melakukan training dengan model baseline 
logistic_reg = LogisticRegression(random_state=0, max_iter=1000).fit(over_x_train, over_x_train)
pred = logistic_reg.predict(x_val)

print(classification_report(y_val, pred))
print("Nilai precision:", precision_score(y_val, pred))
print("Nilai recall:", recall_score(y_val, pred))
print("Nilai F1 score:", f1_score(y_val, pred))
print("Nilai accuracy:",accuracy_score(y_val, pred))

## III.2
Lakukanlah:
1. Eksplorasi _soft voting_, _hard voting_, dan _stacking_.
2. Buatlah model _logistic regression_ dan SVM.
3. Lakukanlah _soft voting_ dari model-model yang dibangun pada poin 2.
4. Lakukan _hard voting_ dari model-model yang dibangun pada poin 2.
5. Lakukanlah _stacking_ dengan _final classifier_ adalah _logistic regression_ dari model-model yang dibangun pada poin 2.
6. Lakukan validasi dengan metrics yang telah ditentukan untuk poin 3, 4, dan 5.

(Tuliskan hasil eksplorasi III.2 poin 1 di sini.)

In [ ]:
# III.2 Kode di sini.

# Eksplorasi soft voting

print("SOFT VOTING\n")

logistic_reg = LogisticRegression(random_state=0)
svm1 = SVC(C=1, probability=True, random_state=0)
svm2 = SVC(C=0.3, probability=True, random_state=0)

soft_voting = VotingClassifier(estimators=[('logistic_reg', logistic_reg), ('svm1', svm1), ('svm2', svm2)], voting='soft')
soft_voting.fit(x_train, y_train)
pred = soft_voting.predict(x_val)

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
cv_scores = cross_validate(soft_voting, x, y, cv=cv, scoring=['precision', 'recall', 'f1', 'accuracy'])

print(f"Mean precision: {np.mean(cv_scores['test_precision'])}")
print(f"Mean recall: {np.mean(cv_scores['test_recall'])}")
print(f"Mean F1: {np.mean(cv_scores['test_f1'])}")
print(f"Mean accuracy: {np.mean(cv_scores['test_accuracy'])}")

print("HARD VOTING\n")
# Eksplorasi hard voting
hard_voting = VotingClassifier(estimators=[('logistic_reg', logistic_reg), ('svm1', svm1), ('svm2', svm2)], voting='hard')
hard_voting.fit(x_train, y_train)
pred = hard_voting.predict(x_val)

print("Nilai precision:", precision_score(y_val, pred))
print("Nilai recall:", recall_score(y_val, pred))
print("Nilai F1 score:", f1_score(y_val, pred))
print("Nilai accuracy", accuracy_score(y_val, pred))

# Cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
cv_scores = cross_validate(hard_voting, x, y, cv=cv, scoring=['precision', 'recall', 'f1', 'accuracy'])

print(f"Mean precision: {np.mean(cv_scores['test_precision'])}")
print(f"Mean recall: {np.mean(cv_scores['test_precision'])}")
print(f"Mean F1: {np.mean(cv_scores['test_f1'])}")
print(f"Mean accuracy: {np.mean(cv_scores['test_accuracy'])}")

# Eksplorasi stacking

print("STACKING\n")
stacking = StackingClassifier(estimators=[('logistic_reg', logistic_reg), ('svm1', svm1), ('svm2', svm2)], final_estimator=LogisticRegression())
stacking.fit(x_train, y_train)
pred = stacking.predict(x_val)

print(classification_report(y_val, pred))
print("Nilai precision:", precision_score(y_val, pred))
print("Nilai recall:", recall_score(y_val, pred))
print("Nilai F1 score", f1_score(y_val, pred))
print("Nilai accuracy",accuracy_score(y_val, pred))

# Cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
cv_scores = cross_validate(stacking, x, y, cv=cv, scoring=['precision', 'recall', 'f1', 'accuracy'])

print(f"Mean precision: {np.mean(cv_scores['test_precision'])}")
print(f"Mean recall: {np.mean(cv_scores['test_precision'])}")
print(f"Mean F1: {np.mean(cv_scores['test_f1'])}")
print(f"Mean accuracy: {np.mean(cv_scores['test_accuracy'])}")

# IV. Analisis
Bandingkan hasil dari hal-hal berikut:
1. Model _baseline_ (II.3)
2. Model lain (II.4)
3. Hasil _undersampling_
4. Hasil _oversampling_
5. Hasil _soft voting_
6. Hasil _hard voting_
7. Hasil _stacking_

1. Model _baseline_ (II.3)
Model baseline berhasil menghasilkan akurasi sebesar 84.85% dan F1-score sebesar 0.53 pada data tes.

2. Model lain (II.4)
B. Model ini memiliki performa yang lebih baik dibandingkan dengan model baseline, dengan akurasi sebesar 85.81% dan F1-score sebesar 0.60 pada data tes. Model ini juga berhasil mengatasi masalah ketidakseimbangan kelas pada dataset.


3. Hasil _undersampling_
Setelah undersampling, performa model meningkat dengan signifikan. Pada model baseline, akurasi meningkat dari 84.85% menjadi 85.29% dan F1-score meningkat dari 0.53 menjadi 0.58. Pada model RandomForestClassifier, akurasi meningkat dari 85.81% menjadi 86.17% dan F1-score meningkat dari 0.60 menjadi 0.65.


4. Hasil _oversampling_
Namun, hasil dari oversampling ini tidak signifikan meningkatkan performa model. Pada model baseline, akurasi meningkat hanya dari 84.85% menjadi 84.95% dan F1-score meningkat dari 0.53 menjadi 0.54. Pada model RandomForestClassifier, akurasi meningkat hanya dari 85.81% 

5. Hasil _soft voting_
Namun, hasil dari oversampling ini tidak signifikan meningkatkan performa model. Pada model baseline, akurasi meningkat hanya dari 84.85% menjadi 84.95% dan F1-score meningkat dari 0.53 menjadi 0.54. Pada model RandomForestClassifier, akurasi meningkat hanya dari 85.81% 

6. Hasil _hard voting_
Hard voting menghasilkan akurasi sebesar 85.27% dan F1-score sebesar 0.59 pada data tes. Hasil ini lebih buruk dibandingkan dengan model baseline dan model RandomForestClassifier.


7. Hasil _stacking_
Stacking menghasilkan akurasi sebesar 85.94% dan F1-score sebesar 0.64 pada data tes. Hasil ini lebih baik dibandingkan dengan model baseline, model RandomForestClassifier, dan hasil hard voting.